# Create island defect on a slab

Create an island defect on a slab

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
# Shape-specific parameters
# Choose the island shape: 'cylinder', 'sphere', 'box', or 'triangular_prism'
# and the corresponding parameters
SHAPE_PARAMETERS = {
    'shape': 'cylinder',
    'radius': 0.25,
    'min_z': 0,
    'max_z': 1
}

# Common parameters
CENTER_POSITION = [0.5, 0.5, 0.5]  # Center of the island
USE_CARTESIAN_COORDINATES = False  # Use Cartesian coordinates for the island
NUMBER_OF_ADDED_LAYERS = 1  # Number of layers to add to the island

# Vacuum parameters for builder
AUTO_ADD_VACUUM = True  # Automatically add vacuum to the slab
VACUUM_THICKNESS = 10.0  # Thickness of the vacuum

# Slab parameters for creating a new slab if provided material is not a slab
DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0,0,1),
    "thickness": 3,
    "vacuum": 5.0,
    "use_orthogonal_z": True,
    "xy_supercell_matrix": [[3, 0], [0, 3]]
}


# Uncomment and set the parameters for other shapes if needed
# SHAPE_PARAMETERS = {
#     'shape': 'sphere',
#     'radius': 0.25
# }
# 
# SHAPE_PARAMETERS = {
#     'shape': 'box',
#     'min_coordinate': [0.25, 0.25, 0],
#     'max_coordinate': [0.75, 0.75, 1]
# }
# 
# SHAPE_PARAMETERS = {
#     'shape': 'triangular_prism',
#     'position_on_surface_1': [0.25, 0.25],
#     'position_on_surface_2': [0.75, 0.25],
#     'position_on_surface_3': [0.5, 0.75],
#     'min_z': 0,
#     'max_z': 1
# }



### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("")

### 1.3. Get input material

In [ ]:
from utils.jupyterlite import get_materials
materials = get_materials(globals())

### 1.4. Preview Slab or create a new one

In [ ]:
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration
from utils.visualize import visualize_materials as visualize

slab = materials[0]
if not slab.metadata or slab.metadata["build"]["configuration"]["type"] != SlabConfiguration.__name__:
    print("The material is not a slab. Creating a new slab...")
    slab_config = SlabConfiguration(
        bulk=materials[0],
        miller_indices=DEFAULT_SLAB_PARAMETERS["miller_indices"],
        thickness=DEFAULT_SLAB_PARAMETERS["thickness"],
        vacuum= DEFAULT_SLAB_PARAMETERS["vacuum"],
        use_orthogonal_z=DEFAULT_SLAB_PARAMETERS["use_orthogonal_z"],
        xy_supercell_matrix=DEFAULT_SLAB_PARAMETERS["xy_supercell_matrix"]
    )
        
    slab = create_slab(slab_config)

visualize([{"material": slab, "title": "Original material"}])
visualize([{"material": slab, "title": "Original material"}], rotation="-90x")

## 2. Create the Target Material
### 2.1. Initialize the defect configuration and builder

In [ ]:
from mat3ra.made.tools.build.defect import IslandSlabDefectConfiguration


island_config = IslandSlabDefectConfiguration.from_dict(
    crystal=slab,
    condition=SHAPE_PARAMETERS,
    number_of_added_layers = NUMBER_OF_ADDED_LAYERS
)

from mat3ra.made.tools.build.defect import IslandSlabDefectBuilder, SlabDefectBuilderParameters
params = SlabDefectBuilderParameters(
    auto_add_vacuum=AUTO_ADD_VACUUM,
    vacuum_thickness=VACUUM_THICKNESS,
)
builder = IslandSlabDefectBuilder(build_parameters=params)

### 2.2. Create the island

In [ ]:
slab_with_island = builder.get_material(island_config)

## 3. Visualize the Result(s)

In [ ]:
visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect ({SHAPE_PARAMETERS['shape']})"}]  ,
rotation="-90x")

visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect ({SHAPE_PARAMETERS['shape']})"}], rotation="-90x")

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(slab_with_island)